In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# getting the name of the directory
# where the this file is present.
current = os.path.dirname(os.path.realpath(os.path.abspath("")))
  
# # adding the parent directory to
# # the sys.path.
sys.path.append(current)

In [4]:
sys.path

['/home/vincent/projetPN89/dev',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/home/vincent/.local/lib/python3.10/site-packages',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/home/vincent/projetPN89']

In [5]:
from random import uniform
import json
import sys
import os
from db import Barrier, Bell, Light, instanciate_captors
from get_data import *

# Database exchanges and log entry addition

In [ ]:
json_file = open("../log.json", "r")
data = json.load(json_file)
json_file.close()
data.update(second_test)
file = open("../log.json","w")
json.dump(data,file, indent=6)
file.close()


# Code à jour

In [6]:
captors_dict = {
      "bar-c100x-abcd1234": {"cur": "", "prev" : "", 
            "associated_light": {"lig-l500X-abcd1234": "", "lig-l500X-efgh2345": ""},
            "associated_bell": {"bel-z200X-abcd1234": ""}
            },
      "bar-c100x-efgh2345": {"cur": "", "prev" : "", 
            "associated_light": {"lig-l500X-ijkl3456": "", "lig-l500X-mnop4567": ""},
            "associated_bell": {"bel-z200X-efgh2345": ""}
            },
      "bar-c100x-ijkl3456": {"cur": "","prev" : "", 
            "associated_light": {"lig-l500X-qrst5678": "", "lig-l500X-uvwx6789": "", "lig-l500X-yzab7890": ""},
            "associated_bell": {"bel-z200X-ijkl3456": ""}
            },
      "bar-c100x-mnop4567": {"cur": "", "prev": "",
            "associated_light": {"lig-l500X-cdef8901": "", "lig-l500X-ghij9012": "", "lig-l500X-klmn0123": ""},
            "associated_bell": {"bel-z200X-mnop4567": ""}
            }
      }

In [7]:
def instanciate_captors ():
   captors = captors_dict
   for barrier in captors.keys():
      captors[barrier]["cur"]= Barrier(device_name=barrier)
      captors[barrier]["prev"]= Barrier(device_name=barrier)
      for light in captors[barrier]["associated_light"]:
         captors[barrier]["associated_light"][light]= Light(device_name=light)
      for bell in captors[barrier]['associated_bell']:
         captors[barrier]['associated_bell'][bell]= Bell(device_name=bell)
   return captors

In [9]:
captors = instanciate_captors()
captors

{'bar-c100x-abcd1234': {'cur': Barrier(device_name='bar-c100x-abcd1234', start_time=0, start_angle=0, end_time=0, end_angle=0, status='inactive'),
  'prev': Barrier(device_name='bar-c100x-abcd1234', start_time=0, start_angle=0, end_time=0, end_angle=0, status='inactive'),
  'associated_light': {'lig-l500X-abcd1234': Light(device_name='lig-l500X-abcd1234', start_time=0, end_time=0),
   'lig-l500X-efgh2345': Light(device_name='lig-l500X-efgh2345', start_time=0, end_time=0)},
  'associated_bell': {'bel-z200X-abcd1234': Bell(device_name='bel-z200X-abcd1234', start_time=0, end_time=0)}},
 'bar-c100x-efgh2345': {'cur': Barrier(device_name='bar-c100x-efgh2345', start_time=0, start_angle=0, end_time=0, end_angle=0, status='inactive'),
  'prev': Barrier(device_name='bar-c100x-efgh2345', start_time=0, start_angle=0, end_time=0, end_angle=0, status='inactive'),
  'associated_light': {'lig-l500X-ijkl3456': Light(device_name='lig-l500X-ijkl3456', start_time=0, end_time=0),
   'lig-l500X-mnop4567': 

In [65]:
for barrier in captors:
    print(barrier)

bar-c100x-abcd1234
bar-c100x-efgh2345
bar-c100x-ijkl3456
bar-c100x-mnop4567


In [67]:
lights_names = [list(captors[barriere]["associated_light"].keys()) for barriere in captors]
lights_names

[['lig-l500X-abcd1234', 'lig-l500X-efgh2345'],
 ['lig-l500X-ijkl3456', 'lig-l500X-mnop4567'],
 ['lig-l500X-qrst5678', 'lig-l500X-uvwx6789', 'lig-l500X-yzab7890'],
 ['lig-l500X-cdef8901', 'lig-l500X-ghij9012', 'lig-l500X-klmn0123']]

In [75]:
associated_barriers = {'lig-l500X-abcd1234' : "bar-c100x-abcd1234",
                       'lig-l500X-efgh2345' : "bar-c100x-abcd1234",
                       'lig-l500X-ijkl3456' :"bar-c100x-efgh2345",
                        'lig-l500X-mnop4567': "bar-c100x-efgh2345",
                        'lig-l500X-qrst5678': "bar-c100x-ijkl3456",
                        'lig-l500X-uvwx6789': "bar-c100x-ijkl3456",
                        'lig-l500X-yzab7890': "bar-c100x-ijkl3456",
                        'lig-l500X-cdef8901': "bar-c100x-mnop4567",
                        'lig-l500X-ghij9012': "bar-c100x-mnop4567",
                        'lig-l500X-klmn0123': "bar-c100x-mnop4567"    
}

In [111]:
def check_barrier_not_down(light_id : str):
    bar_id = associated_barriers.get(light_id)
    bar = captors[bar_id]["cur"]   
    light = captors[bar_id]["associated_light"][light_id]
    if bar.start_time == 0:
        print(f"ENVOYER UN MSG A ELK : Pas de données reçues pour la barrière {bar_id}")
    else :
        if bar.start_time - light.start_time > 8 :
            print("ENVOYER UN AUTRE MSG D'ERREUR A ELK : la barrière n'est pas abaissée à temps")


In [109]:
def barrier_islong(bar_id : str):
    bar = captors[bar_id]['cur']
    if bar.end_time - bar.start_time > 12 :
        print(f"ENCORE UN MSG D'ERREUR : LA BARRIERE {bar_id} EST TRES LONGUE")
    else :
        print ("TOU VA B1 !")

In [112]:
captors["bar-c100x-abcd1234"]['cur'].start_time = 0
captors["bar-c100x-abcd1234"]['cur'].end_time = 29
captors["bar-c100x-abcd1234"]['associated_light']['lig-l500X-abcd1234'].start_time =7

In [113]:
check_barrier_not_down("lig-l500X-abcd1234")

ENVOYER UN MSG A ELK : Pas de données reçues pour la barrière bar-c100x-abcd1234


In [110]:
barrier_islong("bar-c100x-abcd1234")

ENCORE UN MSG D'ERREUR : LA BARRIERE bar-c100x-abcd1234 EST TRES LONGUE


In [10]:
for barrier in captors.keys():
    captors[barrier]["cur"]= Barrier(device_name=barrier)
    captors[barrier]["prev"]= Barrier(device_name=barrier)
    for light in captors[barrier]["associated_light"]:
        captors[barrier]["associated_light"][light]= Light(device_name=light)
    for bell in captors[barrier]['associated_bell']:
        captors[barrier]['associated_bell'][bell]= Bell(device_name=bell)
    

In [58]:
captors["bar-c100x-abcd1234"]["cur"].start_angle = 0
captors["bar-c100x-abcd1234"]["cur"].end_angle = 89

In [54]:
captors["bar-c100x-abcd1234"]

{'cur': Barrier(device_name='bar-c100x-abcd1234', start_time=0, start_angle=1, end_time=0, end_angle=89, status='up'),
 'prev': Barrier(device_name='bar-c100x-abcd1234', start_time=0, start_angle=0, end_time=0, end_angle=0, status='inactive'),
 'associated_light': {'lig-l500X-abcd1234': Light(device_name='lig-l500X-abcd1234', start_time=0, end_time=0),
  'lig-l500X-efgh2345': Light(device_name='lig-l500X-efgh2345', start_time=0, end_time=0)},
 'associated_bell': {'bel-z200X-abcd1234': Bell(device_name='bel-z200X-abcd1234', start_time=0, end_time=0)}}

In [59]:
def update_barrier_status(barrier : Barrier):
    if barrier.start_angle > barrier.end_angle and barrier.end_angle == 0 :
        barrier.change_status("down")
    elif barrier.start_angle < barrier.end_angle and barrier.end_angle == 90 :
        barrier.change_status("up")
    else :
        barrier.change_status("irregular")

In [60]:
update_barrier_status(captors["bar-c100x-abcd1234"]["cur"])
captors["bar-c100x-abcd1234"]

{'cur': Barrier(device_name='bar-c100x-abcd1234', start_time=0, start_angle=0, end_time=0, end_angle=89, status='irregular'),
 'prev': Barrier(device_name='bar-c100x-abcd1234', start_time=0, start_angle=0, end_time=0, end_angle=0, status='inactive'),
 'associated_light': {'lig-l500X-abcd1234': Light(device_name='lig-l500X-abcd1234', start_time=0, end_time=0),
  'lig-l500X-efgh2345': Light(device_name='lig-l500X-efgh2345', start_time=0, end_time=0)},
 'associated_bell': {'bel-z200X-abcd1234': Bell(device_name='bel-z200X-abcd1234', start_time=0, end_time=0)}}

In [95]:
def update_db(captors):
    global new_data
    if new_data:
        for i in new_data[0]:
            id =i['GENERIC_DATA'][0]['device_name'] 
            
            if id.startswith("bar"):
                barrier_pre = captors[id].get("prev")
                barrier_cur = captors[id].get("cur")
                barrier_pre.update_barrier(barrier_cur.__dict__)
                barrier_cur.update_barrier(i['GENERIC_DATA'][0])
                
            if id.startswith("bel"):
                for barrier in captors :
                    bell= captors[barrier]["associated_bell"].get(id)
                    try :
                        bell= captors[barrier]["associated_bell"].get(id)
                        bell.update_bell(i['GENERIC_DATA'][0])
                    except :
                        captors[barrier]["associated_bell"].get(id) == None
                    
            if id.startswith("lig"):
                for barrier in captors:
                    try :
                        light = captors[barrier]["associated_light"].get(id)
                        light.update_light(i['GENERIC_DATA'][0])
                    except :
                        captors[barrier]["associated_light"].get(id) == None

In [96]:
test_bell =[
{
"GENERIC_DATA": [
{
"device_name": "bel-z200X-mnop4567",
"start_time": 111111111,
"end_time": 2222222222
}
]
}
]

In [97]:
update_db(captors, test_bell)

In [98]:
captors

{'bar-c100x-abcd1234': {'cur': Barrier(device_name='bar-c100x-abcd1234', start_time=0, start_angle=0, end_time=0, end_angle=0, status='inactive'),
  'prev': Barrier(device_name='bar-c100x-abcd1234', start_time=0, start_angle=0, end_time=0, end_angle=0, status='inactive'),
  'associated_light': {'lig-l500X-abcd1234': Light(device_name='lig-l500X-abcd1234', start_time=0, end_time=0),
   'lig-l500X-efgh2345': Light(device_name='lig-l500X-efgh2345', start_time=0, end_time=0)},
  'associated_bell': {'bel-z200X-abcd1234': Bell(device_name='bel-z200X-abcd1234', start_time=0, end_time=0)}},
 'bar-c100x-efgh2345': {'cur': Barrier(device_name='bar-c100x-efgh2345', start_time=0, start_angle=0, end_time=0, end_angle=0, status='inactive'),
  'prev': Barrier(device_name='bar-c100x-efgh2345', start_time=0, start_angle=0, end_time=0, end_angle=0, status='inactive'),
  'associated_light': {'lig-l500X-ijkl3456': Light(device_name='lig-l500X-ijkl3456', start_time=0, end_time=0),
   'lig-l500X-mnop4567': 

In [99]:
new_data_bell = [
{
"GENERIC_DATA": [
{
"device_name": "bel-z200X-abcd1234",
"start_time": 111111111,
"end_time" : 33333333333
}
]
}],

[{
"GENERIC_DATA": [
{
"device_name": "bel-z200X-abcd1234",
"start_time": 2222222222,
"end_time" : 5555555555
}
]
}
]

In [ ]:
new_data_barrier = [
{
"GENERIC_DATA": [
{
"device_name": "bar-c100x-abcd1234",
"start_time": 2222222,
"end_time" : 33444444,
"end_angle": 90,
"start_angle" : 0
}
]
}
],

[
{
"GENERIC_DATA": [
{
"device_name": "bar-c100x-abcd1234",
"start_time": 123456789,
"end_time" : 987654321,
"end_angle": 0,
"start_angle" : 90
}
]
}
]

In [ ]:
new_data_light = [
{
"GENERIC_DATA": [
{
"device_name": "lig-l500X-efgh2345",
"start_time": 111111111,
"end_time" : 33333333333
}
]
}],

[{
"GENERIC_DATA": [
{
"device_name": "lig-l500X-abcd1234",
"start_time": 2222222222,
"end_time" : 5555555555
}
]
}
]

In [117]:
new_whole_data =[[
    {
    "GENERIC_DATA": [
                            {
                            "device_name": "lig-l500X-abcd1234",
                            "start_time": 1,
                            "end_time" : 3
                            }
                    ]
    },
    {
    "GENERIC_DATA": [
                            {
                            "device_name": "bar-c100x-abcd1234",
                            "start_time": 111111111,
                            "end_time" : 55555555555,
                            "end_angle": 0,
                            "start_angle" : 90
                            }
                    ]
    }               
]]

In [122]:
new_whole_data

[[{'GENERIC_DATA': [{'device_name': 'lig-l500X-abcd1234',
     'start_time': 1,
     'end_time': 3}]},
  {'GENERIC_DATA': [{'device_name': 'bar-c100x-abcd1234',
     'start_time': 111111111,
     'end_time': 55555555555,
     'end_angle': 0,
     'start_angle': 90}]}]]

In [125]:
for i in new_whole_data[0]:
    id =i['GENERIC_DATA'][0]['device_name']
    print(id)

lig-l500X-abcd1234
bar-c100x-abcd1234


In [221]:
# Instancie la db et les capteurs
db_size = 0
db_get_list = []
captors = instanciate_captors()

In [235]:
# Regular task checking for new data
serv_size = get_serv_size()
check_serv_status(db_size)
db_get_cur = []
# Checking new_data and saving them in log and db for use cases
if check_serv_status(db_size) :
    add_log_entry(get_data()[db_size:])
    db_get_list.append(get_data())
    db_get_cur.append(get_data()[db_size:])
db_size = serv_size

# TEST POUR LOAD/SAVE JSON

In [1]:
json_test = {
        "barrier1": [
            {
            "device_name": "stu-c100x-abcd1234",
            "start_time": 1678123467,
            "start_angle": 83.0,
            "end_time": 1678123482,
            "end_angle": 1.3
            }
        ]
    }


In [7]:
save_file = open("saved_json", "w")
json.dump(json_test, save_file, indent= 6)
save_file.close()

# CREATION DES DICTIONNAIRES PAR TYPE DE CAPTEUR

# Reliquats code précédent

In [ ]:
db_get_list

[[{'GENERIC_DATA': [{'device_name': 'bar-c100x-efgh2345',
     'start_time': 1678123300,
     'end_time': 1678123400,
     'start_angle': 90,
     'end_angle': 0}]}],
 [{'GENERIC_DATA': [{'device_name': 'bar-c100x-efgh2345',
     'start_time': 1678123300,
     'end_time': 1678123400,
     'start_angle': 90,
     'end_angle': 0}]},
  {'GENERIC_DATA': [{'device_name': 'bar-c100x-efgh2345',
     'start_time': 1678123415,
     'end_time': 1678123515,
     'start_angle': 0,
     'end_angle': 90}]}]]

In [ ]:
db_get_cur =   [[{'GENERIC_DATA': [{'device_name': 'bar-c100x-efgh2345',
     'start_time': 1678123415,
     'end_time': 1678123515,
     'start_angle': 0,
     'end_angle': 90}]}]]

In [ ]:
if db_get_cur:
    for i in db_get_cur[0]:
        if i['GENERIC_DATA'][0]['device_name'].startswith("bar"):
            barrier_id = i['GENERIC_DATA'][0]['device_name']
            barrier_pre = captors['captor_list']['barriers'].get(barrier_id+"_pre")
            barrier_cur = captors['captor_list']['barriers'].get(barrier_id+"_cur")
            barrier_pre.update_barrier(barrier_cur.__dict__)
            barrier_cur.update_barrier(i['GENERIC_DATA'][0])
        if i['GENERIC_DATA'][0]['device_name'].startswith("bel"):
            bell_id = i['GENERIC_DATA'][0]['device_name']
            bell_pre = captors['captor_list']['bells'].get(bell_id+"_pre")
            bell_cur = captors['captor_list']['bells'].get(bell_id+"_cur")
            bell_pre.update_bell(bell_cur.__dict__)
            bell_cur.update_bell(i['GENERIC_DATA'][0])
        if i['GENERIC_DATA'][0]['device_name'].startswith("lig"):
            light_id = i['GENERIC_DATA'][0]['device_name']
            light_pre = captors['captor_list']['lights'].get(light_id+"_pre")
            light_cur = captors['captor_list']['lights'].get(light_id+"_cur")
            light_pre.update_barrier(light_cur.__dict__)
            light_cur.update_barrier(i['GENERIC_DATA'][0])


In [ ]:
captors['captor_list']['barriers']

{'bar-c100x-abcd1234_cur': Barrier(device_name='bar-c100x-abcd1234', start_time=0, start_angle=0, end_time=0, end_angle=0, status='inactive'),
 'bar-c100x-abcd1234_pre': Barrier(device_name='bar-c100x-abcd1234', start_time=0, start_angle=0, end_time=0, end_angle=0, status='inactive'),
 'bar-c100x-efgh2345_cur': Barrier(device_name='bar-c100x-efgh2345', start_time=1678123415, start_angle=0, end_time=1678123515, end_angle=90, status='inactive'),
 'bar-c100x-efgh2345_pre': Barrier(device_name='bar-c100x-efgh2345', start_time=1678123300, start_angle=90, end_time=1678123400, end_angle=0, status='inactive'),
 'bar-c100x-ijkl3456_cur': Barrier(device_name='bar-c100x-ijkl3456', start_time=0, start_angle=0, end_time=0, end_angle=0, status='inactive'),
 'bar-c100x-ijkl3456_pre': Barrier(device_name='bar-c100x-ijkl3456', start_time=0, start_angle=0, end_time=0, end_angle=0, status='inactive'),
 'bar-c100x-mnop4567_cur': Barrier(device_name='bar-c100x-mnop4567', start_time=0, start_angle=0, end_ti

In [ ]:
captor_dict = {"captor_list" : {"barrier_list" : 
    ["bar-c100x-abcd1234", "bar-c100x-efgh2345", "bar-c100x-ijkl3456", "bar-c100x-mnop4567"],
"bell_list" : 
    ["bel-z200X-abcd1234", "bel-z200X-efgh2345", "bel-z200X-ijkl3456", "bel-z200X-mnop4567"],
"light_list" : ["lig-l500X-abcd1234", "lig-l500X-efgh2345", "lig-l500X-ijkl3456", "lig-l500X-mnop4567",
    "lig-l500X-qrst5678", "lig-l500X-uvwx6789", "lig-l500X-yzab7890", "lig-l500X-cdef8901"],
"single_lights": ["lig-l500X-ghij9012", "lig-l500X-klmn0123"]}}



In [ ]:
def instanciate_captors ():
   barrier_list = [({f"barrier{num+1}_curr" : Barrier(device_name = captor)}, \
      {f"barrier{num+1}_prec" : Barrier(device_name = captor)}) \
    for num, captor in enumerate(captor_dict['captor_list']['barrier_list'])]

   bell_list = [({f"bell{num+1}_curr" : Bell(device_name = captor)}, \
      {f"bell{num+1}_prec" : Bell(device_name = captor)}) \
      for num, captor in enumerate(captor_dict['captor_list']['bell_list'])]

   light_list = [({f"light{num+1}_curr" : Light(device_name = captor, channel = 2 if num%2 ==1 else 1)}, \
      {f"light{num+1}_prec" : Light(device_name = captor , channel = 2 if num%2 ==1 else 1)}) \
      for num, captor in enumerate(captor_dict['captor_list']['light_list'])]

   light_list.append(({f"light{9}_curr" : Light(device_name = captor_dict['captor_list']['single_lights'][0], channel = 1)},\
      {f"light{9}_prec" : Light(device_name = captor_dict['captor_list']['single_lights'][0], channel = 1)}))
   light_list.append(({f"light{10}_curr" : Light(device_name = captor_dict['captor_list']['single_lights'][1],\
      channel = 1)},{f"light{10}_prec" : Light(device_name = captor_dict['captor_list']['single_lights'][1], channel = 1)}))
   
   return {"captor_list" : {"barrier_list": barrier_list, "bell_list": bell_list, "light_list": light_list}}


In [ ]:
captor_list = instanciate_captors()

In [ ]:
captor_list['captor_list'].keys()

dict_keys(['barrier_list', 'bell_list', 'light_list'])

In [ ]:

for i in fused_data:
    if i[0] == "1":
        for j in range(1,3):
            if i[3] == "True":
                bar_dict[f"barrier {j}"][i[1]] = [{
                    "device_name": "stu-c100x-abcd1234",
                    "start_time": i[2]+8,
                    "start_angle": 90.0 + uniform(-5, 5),
                    "end_time": i[2]+12,
                    "end_angle": 0.0 + uniform(-5,5)
                    }]
                bar_dict[f"barrier {j}"][i[1]].append({
                    "device_name": "stu-c100x-abcd1234",
                    "start_time": i[2]+65,
                    "start_angle": 0.0,
                    "end_time": i[2] +77,
                    "end_angle": 90.0
                    })
            else :
                bar_dict[f"barrier {j}"][i[1]] = [{
                    "device_name": "stu-c100x-abcd1234",
                    "start_time": i[2]+8,
                    "start_angle": 90.0,
                    "end_time": i[2]+12,
                    "end_angle": 0.0
                    }]
                bar_dict[f"barrier {j}"][i[1]].append({
                    "device_name": "stu-c100x-abcd1234",
                    "start_time": i[2]+65,
                    "start_angle": 0.0,
                    "end_time": i[2] +77,
                    "end_angle": 90.0
                    })
    elif i[0] == "2" :
        for j in range(3,5):
            if i[3] == "True":
                bar_dict[f"barrier {j}"][i[1]] = [{
                    "device_name": "stu-c100x-abcd1234",
                    "start_time": i[2]+8,
                    "start_angle": 90.0 + uniform(-5, 5),
                    "end_time": i[2]+12,
                    "end_angle": 0.0 + uniform(-5,5)
                    }]
                bar_dict[f"barrier {j}"][i[1]].append({
                    "device_name": "stu-c100x-abcd1234",
                    "start_time": i[2]+65,
                    "start_angle": 0.0,
                    "end_time": i[2] +77,
                    "end_angle": 90.0
                    })
            bar_dict[f"barrier {j}"][i[1]] = [{
                    "device_name": "stu-c100x-abcd1234",
                    "start_time": i[2]+8,
                    "start_angle": 90.0,
                    "end_time": i[2]+12,
                    "end_angle": 0.0
                    }]
            bar_dict[f"barrier {j}"][i[1]].append({
                "device_name": "stu-c100x-abcd1234",
                "start_time": i[2]+65,
                "start_angle": 0.0,
                "end_time": i[2] +77,
                "end_angle": 90.0
                })
            
            
save_file = open("data_post/barriers.json", "w")
json.dump(bar_dict, save_file, indent= 6)
save_file.close()


In [ ]:
# structure type JSON

[
    {
        "barrier1": [
            {
            "device_name": "stu-c100x-abcd1234",
            "start_time": 1678123467,
            "start_angle": 83.0,
            "end_time": 1678123482,
            "end_angle": 1.3
            }
        ]
    },
    
    {
        "bell1": [
            {
            "device_name": "stu-c100x-abcd1234",
            "start_time": 1678123467,
            "start_angle": 83.0,
            "end_time": 1678123482,
            "end_angle": 1.3
            }
        ]
    },
    
    {
        "light1": [
            {
            "device_name": "stu-c100x-abcd1234",
            "start_time": 1678123467,
            "start_angle": 83.0,
            "end_time": 1678123482,
            "end_angle": 1.3
            }
        ]
    },
    {
        "light2": [
            {
            "device_name": "stu-c100x-abcd1234",
            "start_time": 1678123467,
            "start_angle": 83.0,
            "end_time": 1678123482,
            "end_angle": 1.3
            }
        ]
    }
]

In [169]:
# structure finale JSON PN

json_type= [
    
    {"barriers": 
                {"barrier1": 
                    {},
                "barrier2" : {},
                "barrier3": {},
                "barrier4": {}
                }
            },
    
    {"bells": 
            {"bell1":
            {"device_name": "stu-c100x-abcd1234","start_time": 1678123467,"end_time": 1678123482},
            "bell2": {}
            }
            },
    
    {"lights": 
            {"device_name": "stu-c100x-abcd1234","channel": 1,"start_time": 1678123467,"end_time": 1678123482,
            }}
]

In [ ]:

train_list = ["881235","2256", "881766", "880447", "881235", "2389",\
    "882559", "881766", "880447", "1759", "97653", "2357", "883496", "875963",\
        "882559", "1356", "5562", "881236", "880465", "5573"]
timestamp_list = [1683012695, 1683015972, 1683016276, 1683017804, 1683022304, 1683026642, 1683027371, 1683027911,\
    1683029116, 1683030616, 1683032248, 1683033193, 1683036116, 1683036683, 1683039646, 1683041565, 1683043426,\
        1683045201, 1683047636,1683048370]
pn_list = [str(ele) for ele in [1,2,1,1,2,1,1,2,2,1,1,2,1,1,2,1,2,1,2,1]]
problem_list = [str(ele).replace("0","False").replace("1","True") for ele in [0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0]]

fused_data = list(zip(pn_list, train_list, timestamp_list, problem_list))
fused_data